In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from rtmlib import Body

In [1]:
def safe_kp_to_array(kp, expected_shape):
    """
    Приводит любые выходы модели к np.ndarray формы expected_shape.
    Если ключевые точки не найдены — возвращает нули.

    Parameters
    ----------
    kp : различный тип
        То, что вернула модель (list / tuple / np.ndarray / None).
    expected_shape : tuple[int, int]
        (n_keypoints, 2)

    Returns
    -------
    np.ndarray
        Массив формы expected_shape, dtype=float32.
    """
    # 1. Нет данных → нули
    if kp is None or (isinstance(kp, (list, tuple)) and len(kp) == 0):
        return np.zeros(expected_shape, dtype=np.float32)

    # 2. Модель могла вернуть кортеж (kpts, scores)
    if isinstance(kp, (list, tuple)) and len(kp) == 2 and \
       isinstance(kp[0], (list, np.ndarray)):
        kp = kp[0]

    kp = np.asarray(kp, dtype=np.float32)

    if kp.ndim == 3:       
        kp = kp[0]

    # 4. Оставляем только x, y
    if kp.ndim == 2 and kp.shape[1] > 2:
        kp = kp[:, :2]

    # 5. Приводим к expected_shape
    if kp.shape != expected_shape:
        k_exp, c_exp = expected_shape
        fixed = np.zeros(expected_shape, dtype=np.float32)
        k_cur = min(kp.shape[0], k_exp)
        c_cur = min(kp.shape[1], c_exp)
        fixed[:k_cur, :c_cur] = kp[:k_cur, :c_cur]
        kp = fixed

    return kp


def extract_skeleton_from_video(
    video_path: str,
    model: Body,
    det_frequency: int = 1,
    seq_len: int = 56,
    expected_shape: tuple = (17, 2),
) -> np.ndarray:
    """
    Извлекает последовательность ключевых точек из видео.

    Возвращает np.ndarray формы (seq_len, n_keypoints, 2).
    Если кадров меньше seq_len — дополняет нулями.
    """

    cap = cv2.VideoCapture(video_path)
    frames_data = []

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % det_frequency == 0:
            try:
                keypoints = model(frame)
            except Exception as e:
                print(f"[{os.path.basename(video_path)}] frame {frame_idx}: {e}")
                keypoints = None

            kp = safe_kp_to_array(keypoints, expected_shape)
            frames_data.append(kp)

        frame_idx += 1

    cap.release()

    # Приводим длину к seq_len
    num_frames = len(frames_data)
    if num_frames < seq_len:
        pad = [np.zeros(expected_shape, dtype=np.float32)] * (seq_len - num_frames)
        frames_data.extend(pad)
    elif num_frames > seq_len:
        frames_data = frames_data[:seq_len]

    return np.array(frames_data, dtype=np.float32)

def process_video(
    video_path: str,
    model: Body,
    output_dir: str,
    label: str,
    det_frequency: int,
    seq_len: int,
    expected_shape: tuple,
):
    """
    Извлекает скелет, сохраняет в .npy и возвращает запись для DataFrame.
    """
    skeleton = extract_skeleton_from_video(
        video_path,
        model,
        det_frequency,
        seq_len,
        expected_shape,
    )
    base = os.path.splitext(os.path.basename(video_path))[0]
    out_file = os.path.join(output_dir, base + "_skeleton.npy")
    np.save(out_file, skeleton)

    return {
        "video_path": video_path,
        "skeleton_file": out_file,
        "num_frames": len(skeleton),
        "label": label,
    }

def process_all_videos_parallel(
    base_dir: str,
    output_dir: str,
    model: Body,
    label: str,
    det_frequency: int = 1,
    seq_len: int = 56,
    expected_shape: tuple = (17, 2),
    max_workers: int = 4,
) -> pd.DataFrame:
    """
    Рекурсивно ищет .mp4/.avi в base_dir и обрабатывает их в ThreadPool.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    video_files = [
        os.path.join(root, f)
        for root, _, files in os.walk(base_dir)
        for f in files
        if f.lower().endswith((".mp4", ".avi"))
    ]

    records = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(
                process_video,
                vp,
                model,
                output_dir,
                label,
                det_frequency,
                seq_len,
                expected_shape,
            )
            for vp in video_files
        ]
        for fut in as_completed(futures):
            try:
                records.append(fut.result())
            except Exception as e:
                print(f"✖ Ошибка обработки видео: {e}")

    return pd.DataFrame(records)


SEQ_LEN = 4
EXPECTED_SHAPE = (17, 2)          
DEVICE = "cuda"                  

# Папки с видео
BASE_TRAIN_DIR = r"C:\Users\jet\Desktop\видосы\norm_v_processed_3_0_frame\train"
OUTPUT_TRAIN_DIR = r"C:\Users\jet\Desktop\видосы\np\2vr_10\skeletons_train"

BASE_VAL_DIR = r"C:\Users\jet\Desktop\видосы\norm_v_processed_3_0_frame\val"
OUTPUT_VAL_DIR = r"C:\Users\jet\Desktop\видосы\np\2vr_10\skeletons_val"

CLASS_DIRS = ["false", "man_doing_something", "true"]  # имена подпапок-классов
MAX_WORKERS = 4
DET_FREQUENCY = 1


def main():
    # Загружаем модель один раз
    model = Body(mode="lightweight", backend="onnxruntime", device=DEVICE)

    # ------------------- Обучающая выборка -------------------
    train_dfs = []
    for cls in CLASS_DIRS:
        cls_dir = os.path.join(BASE_TRAIN_DIR, cls)
        print(f"▶ Train | класс «{cls}»")
        df_cls = process_all_videos_parallel(
            cls_dir,
            OUTPUT_TRAIN_DIR,
            model,
            label=cls,
            det_frequency=DET_FREQUENCY,
            seq_len=SEQ_LEN,
            expected_shape=EXPECTED_SHAPE,
            max_workers=MAX_WORKERS,
        )
        train_dfs.append(df_cls)

    df_train = pd.concat(train_dfs, ignore_index=True)
    train_csv = os.path.join(OUTPUT_TRAIN_DIR, "skeleton_records_train.csv")
    df_train.to_csv(train_csv, index=False)
    print(f"✓ Сохранён train-CSV: {train_csv}")

    # ------------------- Валидационная выборка -------------------
    val_dfs = []
    for cls in CLASS_DIRS:
        cls_dir = os.path.join(BASE_VAL_DIR, cls)
        print(f"▶ Val   | класс «{cls}»")
        df_cls = process_all_videos_parallel(
            cls_dir,
            OUTPUT_VAL_DIR,
            model,
            label=cls,
            det_frequency=DET_FREQUENCY,
            seq_len=SEQ_LEN,
            expected_shape=EXPECTED_SHAPE,
            max_workers=MAX_WORKERS,
        )
        val_dfs.append(df_cls)

    df_val = pd.concat(val_dfs, ignore_index=True)
    val_csv = os.path.join(OUTPUT_VAL_DIR, "skeleton_records_val.csv")
    df_val.to_csv(val_csv, index=False)
    print(f"✓ Сохранён val-CSV: {val_csv}")


if __name__ == "__main__":
    main()


load C:\Users\jet\.cache\rtmlib\hub\checkpoints\yolox_tiny_8xb8-300e_humanart-6f3252f9.onnx with onnxruntime backend
load C:\Users\jet\.cache\rtmlib\hub\checkpoints\rtmpose-s_simcc-body7_pt-body7_420e-256x192-acd4a1ef_20230504.onnx with onnxruntime backend
▶ Train | класс «false»
▶ Train | класс «man_doing_something»
▶ Train | класс «true»
✓ Сохранён train-CSV: C:\Users\jet\Desktop\видосы\np\2vr_10\skeletons_train\skeleton_records_train.csv
▶ Val   | класс «false»
▶ Val   | класс «man_doing_something»
▶ Val   | класс «true»
✓ Сохранён val-CSV: C:\Users\jet\Desktop\видосы\np\2vr_10\skeletons_val\skeleton_records_val.csv
